In [468]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import os
import glob
from functools import partial
import seaborn as sns

plt.style.use('ggplot')

In [5]:
!pwd

/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1


### Combine all CSVs for LBJ and MJ reg season stats

In [35]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/Michael")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)
combined_csv.to_csv("combined_Michael.csv", index=False)

In [41]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/Lebron")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)
combined_csv.to_csv("combined_Lebron.csv", index=False)

In [345]:
mj_df = pd.read_csv('data/mj_comb_szn_stats_cleaned.csv')

In [346]:
lbj_df = pd.read_csv('data/lbj_comb_szn_stats_cleaned.csv')

### Drop Unneccessary Columns from lbj_df and mj_df

### Clean up and format LBJ and MJ df's
- Clean up rows that are repeated headers
- Convert `Date` column into datetime format
- Drop `Unnamed: 0` column from both df's
- Drop other unneccessary columns from lbj_df and mj_df

In [347]:
lbj_df['Date'] = lbj_df['Date'].apply(pd.to_datetime, format='%Y-%m-%d')
mj_df['Date'] = mj_df['Date'].apply(pd.to_datetime, format='%Y-%m-%d')

In [348]:
lbj_df.sort_values('Date', inplace=True)

In [349]:
#DO NOT RUN AGAIN

mj_df.rename(columns={mj_df.columns[0]: "drop1"}, inplace=True)
lbj_df.rename(columns={lbj_df.columns[0]: "drop1"}, inplace=True)

In [352]:
print(lbj_df.columns)
print(mj_df.columns)

Index(['drop1', 'G', 'Date', 'Age', 'WL', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PTS', 'GmSc', '+/-'],
      dtype='object')
Index(['drop1', 'G', 'Date', 'Age', 'WL', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'TRB', 'AST', 'STL', 'BLK', 'TOV',
       'PTS', 'GmSc', '+/-'],
      dtype='object')


In [353]:
#NaN column
mj_df.drop(columns=['drop1', 'G', 'MP', '+/-', '3P%', 'FT%'], inplace=True)
lbj_df.drop(columns=['drop1', 'G', 'MP', '+/-', '3P%', 'FT%'], inplace=True)

In [358]:
lbj_cols = list(lbj_df.columns)
print(lbj_cols)
    
num_cols = ['FG', 'Age', 'FGA', 'FG%', '3P', '3PA', 'FT',
            'FTA', 'TRB', 'AST', 'STL', 'BLK',
            'TOV', 'PTS', 'GmSc']

lbj_df[num_cols] = lbj_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)
mj_df[num_cols] = mj_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)

['Date', 'Age', 'WL', 'FG', 'FGA', 'FG%', '3P', '3PA', 'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'GmSc']


In [359]:
#Checking for rows with NaN values

is_NaN = mj_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = mj_df[row_has_NaN]
print(rows_with_NaN)

is_NaN = lbj_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = lbj_df[row_has_NaN]
print(rows_with_NaN)

Empty DataFrame
Columns: [Date, Age, WL, FG, FGA, FG%, 3P, 3PA, FT, FTA, TRB, AST, STL, BLK, TOV, PTS, GmSc]
Index: []
Empty DataFrame
Columns: [Date, Age, WL, FG, FGA, FG%, 3P, 3PA, FT, FTA, TRB, AST, STL, BLK, TOV, PTS, GmSc]
Index: []


###### GmSc - Game Score; the formula is PTS + 0.4 * FG - 0.7 * FGA - 0.4*(FTA - FT) + 0.7 * ORB + 0.3 * DRB + STL + 0.7 * AST + 0.7 * BLK - 0.4 * PF - TOV. Game Score was created by John Hollinger to give a rough measure of a player's productivity for a single game. The scale is similar to that of points scored, (40 is an outstanding performance, 10 is an average performance, etc.).

#### Save cleaned data to CSV (lbj_df and mj_df)

In [360]:
lbj_df.to_csv("final_lbj_comb_szn_stats.csv")
mj_df.to_csv("final_mj_comb_szn_stats.csv")

### Postseason Stats for Lebron and Michael
- Change "{year} Playoffs" column to "Date"
- Remove rows that are repeated headers
- Convert applicable columns into `numeric`, date into `datetime` format

In [413]:
lbj_ps_df = pd.read_csv("data/Lebron_postseason.csv")
mj_ps_df = pd.read_csv("data/Michael_postseason.csv")

In [414]:
print("LBJ", lbj_ps_df.columns)
print("MJ", mj_ps_df.columns)

lbj_ps_df.info()

LBJ Index(['Unnamed: 0', 'Rk', 'G', '2006 Playoffs', 'Series', 'Tm', ' ', 'Opp',
       'G#', ' .1', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'GmSc', '+/-'],
      dtype='object')
MJ Index(['Unnamed: 0', 'Rk', 'G', '1985 Playoffs', 'Series', 'Tm', ' ', 'Opp',
       'G#', ' .1', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT',
       'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'GmSc'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 32 columns):
Unnamed: 0       283 non-null int64
Rk               251 non-null object
G                251 non-null object
2006 Playoffs    251 non-null object
Series           251 non-null object
Tm               251 non-null object
                 130 non-null object
Opp              251 non-null object
G#               251 non-null obje

In [415]:
#gets rid of first 2 columns which are just index columns, gets rid of unnecessary columns

mj_ps_df.rename(columns={mj_ps_df.columns[6]: "drop1", mj_ps_df.columns[9]: "WL"}, inplace=True)
lbj_ps_df.rename(columns={lbj_ps_df.columns[6]: 'drop1', lbj_ps_df.columns[9]: "WL"}, inplace=True)

In [416]:
lbj_ps_df.drop(columns=lbj_ps_df.columns[0:2], inplace=True)
lbj_ps_df.drop(columns=['Tm', 'Opp', 'G#', 'GS', 'MP', 'ORB', 'DRB', 'PF',
                        'G', 'Series', 'drop1','3P%', 'FT%'], inplace=True)

mj_ps_df.drop(columns=mj_ps_df.columns[0:2], inplace=True)
mj_ps_df.drop(columns=['Tm', 'Opp', 'G#', 'GS', 'MP', 'ORB', 'DRB', 'PF',
                      'G', 'Series', 'drop1','3P%', 'FT%'], inplace=True)

lbj_ps_df = lbj_ps_df.rename(columns={'2006 Playoffs': "Date"})
mj_ps_df = mj_ps_df.rename(columns={"1985 Playoffs": "Date"})

#Get rid of empty rows
lbj_ps_df = lbj_ps_df[pd.notnull(lbj_ps_df["Date"])]
mj_ps_df = mj_ps_df[pd.notnull(mj_ps_df["Date"])]

#Convert "WL" column into a simple format: W or L
lbj_ps_df["WL"] = lbj_ps_df.apply(lambda x: x["WL"][0], axis=1)
mj_ps_df["WL"] = mj_ps_df.apply(lambda x: x["WL"][0], axis=1)

In [421]:
#Turn values to numeric and get rid of rows that are repeated headers

lbj_ps_df = lbj_ps_df[lbj_ps_df.FT != 'FT']
mj_ps_df = mj_ps_df[mj_ps_df.FT != 'FT']
    
num_cols_lbj = ['FG', 'FGA', 'FG%', '3P', '3PA', 'FT', 'FTA',
            'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'GmSc', '+/-']

#mj does not have the '+/-' column
num_cols_mj = ['FG', 'FGA', 'FG%', '3P', '3PA', 'FT', 'FTA',
            'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'GmSc']

lbj_ps_df[num_cols_lbj] = lbj_ps_df[num_cols_lbj].apply(pd.to_numeric, errors='ignore', axis=1)
mj_ps_df[num_cols_mj] = mj_ps_df[num_cols_mj].apply(pd.to_numeric, errors='ignore', axis=1)

lbj_ps_df['Date'] = lbj_ps_df['Date'].apply(pd.to_datetime, format='%Y-%m-%d')
mj_ps_df['Date'] = mj_ps_df['Date'].apply(pd.to_datetime, format='%Y-%m-%d')

['Date', 'WL', 'FG', 'FGA', 'FG%', '3P', '3PA', 'FT', 'FTA', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'GmSc', '+/-']


/Users/edwardkim/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/edwardkim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/edwardkim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [425]:
#Check for rows in ps_df's

is_NaN = mj_ps_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = mj_ps_df[row_has_NaN]
print("MJ_PS", rows_with_NaN)

is_NaN = lbj_ps_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = lbj_ps_df[row_has_NaN]
print("LBJ_PS",rows_with_NaN)

MJ_PS Empty DataFrame
Columns: [Date, WL, FG, FGA, FG%, 3P, 3PA, FT, FTA, TRB, AST, STL, BLK, TOV, PTS, GmSc]
Index: []
LBJ_PS Empty DataFrame
Columns: [Date, WL, FG, FGA, FG%, 3P, 3PA, FT, FTA, TRB, AST, STL, BLK, TOV, PTS, GmSc, +/-]
Index: []


In [426]:
lbj_ps_df.to_csv("final_lbj_ps_cleaned.csv")
mj_ps_df.to_csv("final_mj_ps_cleaned.csv")

## Advanced Stats and Reg Season Stats For All Players

#### Add "Year" column to each season csv to be able to differentiate duplicate player entries
- Sort file list so that the correct year is assigned to each CSV
- Once column is added, save as new csv and then concatenate into csv --> pd df

In [57]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/advanced_stats")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames.sort()
print(all_filenames)
print(len(all_filenames))

years = list(range(1985, 1994))
years.extend(list(range(1995, 1999)))
years.extend(list(range(2002, 2020)))
print(years)
print(len(years))

for i,j in zip(all_filenames, years):
    df = pd.read_csv(i)
    df["Year"] = int(j)
    df.to_csv(f"year_col_added_{i}")

['advanced_all_players_1985.csv', 'advanced_all_players_1986.csv', 'advanced_all_players_1987.csv', 'advanced_all_players_1988.csv', 'advanced_all_players_1989.csv', 'advanced_all_players_1990.csv', 'advanced_all_players_1991.csv', 'advanced_all_players_1992.csv', 'advanced_all_players_1993.csv', 'advanced_all_players_1995.csv', 'advanced_all_players_1996.csv', 'advanced_all_players_1997.csv', 'advanced_all_players_1998.csv', 'advanced_all_players_2002.csv', 'advanced_all_players_2003.csv', 'advanced_all_players_2004.csv', 'advanced_all_players_2005.csv', 'advanced_all_players_2006.csv', 'advanced_all_players_2007.csv', 'advanced_all_players_2008.csv', 'advanced_all_players_2009.csv', 'advanced_all_players_2010.csv', 'advanced_all_players_2011.csv', 'advanced_all_players_2012.csv', 'advanced_all_players_2013.csv', 'advanced_all_players_2014.csv', 'advanced_all_players_2015.csv', 'advanced_all_players_2016.csv', 'advanced_all_players_2017.csv', 'advanced_all_players_2018.csv', 'advanced

### Concatenate the CSVs and read them into PD DF

In [513]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/reg_season_stats")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)
combined_csv.to_csv("reg_szn_85-19.csv", index=False)

In [514]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/data/advanced_stats")

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames], sort=False)
combined_csv.to_csv("adv_szn_85-19.csv", index=False)

In [516]:
os.chdir("/Users/edwardkim/Documents/Galvanize/DSI/NBA-LBJ-vs-MJ-Capstone1/")

In [517]:
adv_df = pd.read_csv('data/advanced_stats/adv_szn_85-19.csv')

In [518]:
szn_df = pd.read_csv('data/reg_season_stats/reg_szn_85-19.csv')

### Convert applicable rows to numeric

In [519]:
adv_cols = adv_df.columns
adv_cols
    
num_cols = ['Age','G', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 
            'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
            'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

adv_df[num_cols] = adv_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)

#gets rid of first 2 columns that are just indices
adv_df.drop(columns=adv_df.columns[:2], inplace=True)

### Remove rows that are repeated headers and unneccessary columns

In [520]:
szn_cols = szn_df.columns

num_cols = ['Age','G', 'GS', 'MP', 'FG',
            'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT',
            'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
            'PTS']

szn_df[num_cols] = szn_df[num_cols].apply(pd.to_numeric, errors='coerce', axis=1)

#gets rid of first 2 columns that are just indices
szn_df.drop(columns=szn_df.columns[:2], inplace=True)

In [521]:
szn_drop_cols = ['Tm', 'GS', 'MP', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB','PF']
adv_drop_cols = ['MP','3PAr', 'FTr','ORB%', 'DRB%','OBPM', 'DBPM']

szn_df.drop(columns=szn_drop_cols, inplace=True)
adv_df.drop(columns=adv_drop_cols, inplace=True)

In [533]:
szn_df.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'G', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '2P', '2PA', '2P%', 'eFG%', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS',
       'Year'],
      dtype='object')

In [524]:
# adv_df.drop(columns=adv_df.columns[-4], inplace=True)

In [528]:
# adv_df.drop(columns=adv_df.columns[-8], inplace=True)

In [530]:
adv_df = adv_df[adv_df.Rk != 'Rk']
szn_df = szn_df[szn_df.Rk != 'Rk']

In [531]:
szn_df.head(3)

,Rk,Player,Pos,Age,G,FG,FGA,FG%,3P,3PA,...,2PA,2P%,eFG%,TRB,AST,STL,BLK,TOV,PTS,Year
0,1,Kareem Abdul-Jabbar*,C,41.0,74.0,4.2,8.9,0.475,0.0,0.0,...,8.9,0.477,0.475,4.5,1.0,0.5,1.1,1.3,10.1,1989
1,2,Mark Acres,C,26.0,62.0,0.9,1.8,0.482,0.0,0.0,...,1.8,0.478,0.487,2.4,0.3,0.3,0.1,0.4,2.2,1989
2,3,Michael Adams,PG,26.0,77.0,6.1,14.1,0.433,2.2,6.1,...,8.0,0.490,0.509,3.7,6.4,2.2,0.1,2.3,18.5,1989


In [532]:
#Save cleaned datasets as csv

adv_df.to_csv("final_adv_data_cleaned.csv")
szn_df.to_csv("final_szn_data_cleaned.csv")